In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import os
from glob import glob

import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
import xarray as xr

from dask.distributed import Client
from IPython.display import Image, HTML

In [3]:
from icenet.data.sic.mask import Masks

Load data from numpy files

In [4]:
root = "../pipeline"
name = "sipn_test_0_south_forecast"
dates = [dt.date(*[2022, 12, 1])]

In [5]:
def get_prediction_data_nb(root: object, name: object, date: object) -> tuple:
    """
    Based on IceNet library v0.2.7.
    
    Ref:
    https://github.com/icenet-ai/icenet/blob/cb1cb785808ba1138c0ed0f8f88208a144daa6ff/icenet/process/predict.py#L63-L89
    """
    glob_str = os.path.join(root, "results", "predict", name, "*",
                            date.strftime("%Y_%m_%d.npy"))

    np_files = glob(glob_str)
    if not len(np_files):
        logging.warning("No files found")
        return None

    data = [np.load(f) for f in np_files]
    data = np.array(data)
    ens_members = data.shape[0]

    logging.debug("Data read from disk: {} from: {}".format(
        data.shape, np_files))

    return np.stack([data.mean(axis=0), data.std(axis=0)],
                    axis=-1).squeeze(), data, ens_members

In [6]:
arr, data, ens_members = get_prediction_data_nb(root, name, dates[0])

# Make sure we're only working with the 90 days needed for the SIPN South submission.
data = data[..., :90]

print("Number of ensemble members:", ens_members)

Number of ensemble members: 10


Load ensemble mean + stddev data from final netcdf output

In [7]:
ds = xr.open_dataset("../pipeline/results/predict/sipn_test_0_south_forecast.nc")
land_mask = Masks(south=True, north=False).get_land_mask()

# Dimensions for all data being inserted into xarray
data_dims_list = ["time", "yc", "xc", "leadtime"]

# Dict to store all ensemble results (for creating an xarray)
sic_data = {}

# Apply np.nan to land mask regions (emulating icenet output)
land_mask_nan = land_mask.astype(float)
land_mask_nan[land_mask] = np.nan
land_mask_nan[~land_mask] = 1.0

for ensemble in range(ens_members):
    sic_data[f"sic_{ensemble}"] = (data_dims_list, data[ensemble]*land_mask_nan[np.newaxis, :, :, np.newaxis])


data_vars=dict(
    Lambert_Azimuthal_Grid=ds.Lambert_Azimuthal_Grid.data,
    # Extract just the first date (incase more have been predicted)
    # and up to 90 forecast days.
    sic_mean=(data_dims_list, ds.sic_mean[:1, ..., :90].data),
    sic_stddev=(data_dims_list, ds.sic_stddev[:1, ..., :90].data),
    ensemble_members=(["time"], [ens_members]),
)

data_vars.update(sic_data)

Convert all ensemble prediction data to Xarray dataset, using ensemble mean netCDF file as template

In [8]:
# Update longitude range to be 0 to 360 instead of -180 to 180
# makes life easier for Diagnostic 2.
longitude = ds.lon.data.copy()
longitude[longitude<0] += 360

xarr = xr.Dataset(
    data_vars=data_vars,
    coords=dict(
        time=[ds.time[1].data],
        leadtime=ds.leadtime[:90].data,
        forecast_date=ds.forecast_date[1][:90].data,
        xc=ds.xc.data,
        yc=ds.yc.data,
        lat=(("yc", "xc"), ds.lat.data),
        lon=(("yc", "xc"), longitude),
    )
)

xarr

<xarray.Dataset>
Dimensions:                 (time: 1, yc: 432, xc: 432, leadtime: 90,
                             forecast_date: 90)
Coordinates:
  * time                    (time) datetime64[ns] 2022-12-01
  * leadtime                (leadtime) int64 1 2 3 4 5 6 7 ... 85 86 87 88 89 90
  * forecast_date           (forecast_date) datetime64[ns] 2022-12-02 ... 202...
  * xc                      (xc) float64 -5.388e+03 -5.362e+03 ... 5.388e+03
  * yc                      (yc) float64 5.388e+03 5.362e+03 ... -5.388e+03
    lat                     (yc, xc) float32 -16.62 -16.82 ... -16.82 -16.62
    lon                     (yc, xc) float32 315.0 315.1 315.3 ... 135.1 135.0
Data variables: (12/14)
    Lambert_Azimuthal_Grid  int32 -2147483647
    sic_mean                (time, yc, xc, leadtime) float32 nan nan ... nan nan
    sic_stddev              (time, yc, xc, leadtime) float32 nan nan ... nan nan
    ensemble_members        (time) int64 10
    sic_0                   (time, yc, xc, leadtime) float64 nan nan ... nan nan
    sic_1                   (time, yc, xc, leadtime) float64 nan nan ... nan nan
    ...                      ...
    sic_4                   (time, yc, xc, leadtime) float64 nan nan ... nan nan
    sic_5                   (time, yc, xc, leadtime) float64 nan nan ... nan nan
    sic_6                   (time, yc, xc, leadtime) float64 nan nan ... nan nan
    sic_7                   (time, yc, xc, leadtime) float64 nan nan ... nan nan
    sic_8                   (time, yc, xc, leadtime) float64 nan nan ... nan nan
    sic_9                   (time, yc, xc, leadtime) float64 nan nan ... nan nan

Define functions to compute sea ice area and binned sea ice area.

In [167]:
def sic_compute_np(sea_ice_concentration, grid_cell_area=25*25, threshold=0.15, plot=False):
    sic = sea_ice_concentration

    # Mask values less than the threshold
    sic = ma.masked_where(sic<threshold, sic)

    # Multiply by grid-cell area
    sic *= 25*25

    print(sic)
    # Divide by 10^6 due to SIPN South unit requirement
    # (i.e., units must be in 10^6 km^2)
    sea_ice_area = np.sum(sic) / 1E6


    return sea_ice_area

def compute_sea_ice_area_binned_np(sea_ice_concentration, *args, **kwargs):
    sic = sea_ice_concentration

    longitude_bin = xr.DataArray(np.linspace(0, 360, 36+1))

    sea_ice_area_binned = sic.groupby_bins("lon", longitude_bin).map(sic_compute_np, args=args, **kwargs)

    return sea_ice_area_binned

In [137]:
def sic_compute(sea_ice_concentration, grid_cell_area=25*25, threshold=0.15, plot=False):
    sic = sea_ice_concentration

    # Mask values less than the threshold
    sic = sic.where(sic>0.15)

    # Multiply by grid-cell area
    sic *= 25*25

    # Divide by 10^6 due to SIPN South unit requirement
    # (i.e., units must be in 10^6 km^2)
    sea_ice_area = sic.sum() / 1E6

    return sea_ice_area

def compute_sea_ice_area_binned(sea_ice_concentration, *args, **kwargs):
    sic = sea_ice_concentration

    longitude_bin = xr.DataArray(np.linspace(0, 360, 36+1))

    sea_ice_area_binned = sic.groupby_bins("lon", longitude_bin).map(sic_compute, args=args, **kwargs)

    print(sea_ice_area_binned)

    return sea_ice_area_binned

# def compute_sea_ice_area_binned_dask(sea_ice_concentration):


In [94]:
sea_ice_area = xarr[f"sic_{0}"].isel(leadtime=0).map_blocks(sic_compute)

sea_ice_area

<xarray.DataArray 'sic_0' ()>
array(9.06379805)
Coordinates:
    leadtime  int64 1

In [109]:
# Define a template DataArray for what's being returned to map_blocks.
ret_array = xr.DataArray(
    np.zeros((36)),
    coords={"leadtime": np.zeros((36))},
).chunk(5)

ret_array.chunksizes

Frozen({'leadtime': (5, 5, 5, 5, 5, 5, 5, 1)})

In [60]:
xarr[f"sic_{0}"].chunk({"leadtime": 5}).groupby_bins("lon", longitude_bin)

DataArrayGroupBy, grouped over 'lon_bins'
36 groups with labels (310.0,, 320.0], ..., (150.0,, ....

In [123]:
xarr_chunk = xarr.chunk({"leadtime": -1})

In [ ]:
# sea_ice_area_binned = xarr[f"sic_{0}"].chunk({"leadtime": 5}).map_blocks(compute_sea_ice_area_binned, template=ret_array).compute()

# sea_ice_area_binned = xarr[f"sic_{0}"].map_blocks(compute_sea_ice_area_binned).compute()
# sea_ice_area_binned = xr.apply_ufunc(compute_sea_ice_area_binned, xarr[f"sic_{0}"].chunk({"leadtime": 5}), dask="parallelized", output_dtypes=[float])


# longitude_bin = xr.DataArray(np.linspace(0, 360, 36+1))
# sea_ice_area_binned = xr.apply_ufunc(sic_compute_np,
#                                      xarr[f"sic_{0}"].isel(leadtime=0).groupby_bins("lon", longitude_bin),
#                                     #  input_core_dims=[["lon_bins"]],
#                                      output_core_dims=[["dummy"]],
#                                      dask="parallelized",
#                                      output_dtypes=[float]
#                                      )
# sea_ice_area_binned.compute()

sea_ice_area_binned = xarr_chunk[f"sic_{0}"].map_blocks(compute_sea_ice_area_binned, template=ret_array)

sea_ice_area_binned

In [118]:
sea_ice_area_binned = np.asarray([xarr.isel(leadtime=day-1)[f"sic_{0}"].map_blocks(compute_sea_ice_area_binned) for day in xarr.leadtime])

<xarray.DataArray 'sic_0' (lon_bins: 36)>
array([0.34327669, 0.3839828 , 0.43273046, 0.29784033, 0.15688105,
       0.07072017, 0.13614046, 0.17237017, 0.07775545, 0.08777086,
       0.09086775, 0.08306254, 0.09707625, 0.07262376, 0.03565898,
       0.0870581 , 0.20366686, 0.31378991, 0.32147637, 0.39611291,
       0.38858803, 0.3685207 , 0.51234148, 0.42099894, 0.19957403,
       0.16657304, 0.18682177, 0.03583466, 0.02015961, 0.02886602,
       0.46195762, 0.502402  , 0.49096155, 0.5194478 , 0.47865723,
       0.42123173])
Coordinates:
  * lon_bins  (lon_bins) object (0.0, 10.0] (10.0, 20.0] ... (350.0, 360.0]
    leadtime  int64 1
<xarray.DataArray 'sic_0' (lon_bins: 36)>
array([0.33502117, 0.37432696, 0.42631134, 0.28953455, 0.14581124,
       0.06775585, 0.12970424, 0.16246567, 0.0759885 , 0.08540233,
       0.08905887, 0.08069495, 0.09497735, 0.07231736, 0.03546305,
       0.08446485, 0.20015638, 0.3092064 , 0.31852199, 0.38964366,
       0.37511226, 0.35022513, 0.48779807, 0.402

In [138]:
import dask.array as da

sea_ice_area_binned = np.asarray([da.map_blocks(compute_sea_ice_area_binned, xarr_chunk[f"sic_{0}"], dtype=xarr_chunk[f"sic_{0}"].dtype).compute() for day in xarr.leadtime])

In [127]:
xarr_chunk[f"sic_{0}"][:, :, :, 0]

<xarray.DataArray 'sic_0' (time: 1, yc: 432, xc: 432)>
dask.array<getitem, shape=(1, 432, 432), dtype=float64, chunksize=(1, 432, 432), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 2022-12-01
    leadtime  int64 1
  * xc        (xc) float64 -5.388e+03 -5.362e+03 ... 5.362e+03 5.388e+03
  * yc        (yc) float64 5.388e+03 5.362e+03 ... -5.362e+03 -5.388e+03
    lat       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
    lon       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>

In [173]:
xarr

<xarray.Dataset>
Dimensions:                 (time: 1, yc: 432, xc: 432, leadtime: 90,
                             forecast_date: 90)
Coordinates:
  * time                    (time) datetime64[ns] 2022-12-01
  * leadtime                (leadtime) int64 1 2 3 4 5 6 7 ... 85 86 87 88 89 90
  * forecast_date           (forecast_date) datetime64[ns] 2022-12-02 ... 202...
  * xc                      (xc) float64 -5.388e+03 -5.362e+03 ... 5.388e+03
  * yc                      (yc) float64 5.388e+03 5.362e+03 ... -5.388e+03
    lat                     (yc, xc) float32 -16.62 -16.82 ... -16.82 -16.62
    lon                     (yc, xc) float32 315.0 315.1 315.3 ... 135.1 135.0
Data variables: (12/14)
    Lambert_Azimuthal_Grid  int32 -2147483647
    sic_mean                (time, yc, xc, leadtime) float32 nan nan ... nan nan
    sic_stddev              (time, yc, xc, leadtime) float32 nan nan ... nan nan
    ensemble_members        (time) int64 10
    sic_0                   (time, yc, xc, leadtime) float64 nan nan ... nan nan
    sic_1                   (time, yc, xc, leadtime) float64 nan nan ... nan nan
    ...                      ...
    sic_4                   (time, yc, xc, leadtime) float64 nan nan ... nan nan
    sic_5                   (time, yc, xc, leadtime) float64 nan nan ... nan nan
    sic_6                   (time, yc, xc, leadtime) float64 nan nan ... nan nan
    sic_7                   (time, yc, xc, leadtime) float64 nan nan ... nan nan
    sic_8                   (time, yc, xc, leadtime) float64 nan nan ... nan nan
    sic_9                   (time, yc, xc, leadtime) float64 nan nan ... nan nan

In [141]:
xarr[f"sic_{0}"].isel(leadtime=0).groupby_bins("lon", longitude_bin).chunk({"leadtime": 5})

AttributeError: 'DataArrayGroupBy' object has no attribute 'chunk'

In [185]:
# longitude_bin = xr.DataArray(np.linspace(0, 360, 36+1))
# sia_bin = xarr[f"sic_{0}"].groupby_bins("lon", longitude_bin).map(
#                             sic_compute,
#                             # keep_attrs=True,
#                         )

import dask.array as da

sea_ice_area_binned = np.asarray([da.map_blocks(compute_sea_ice_area_binned, xarr_chunk[f"sic_{0}"].isel(leadtime=day-1), dtype=xarr_chunk[f"sic_{0}"].dtype) for day in xarr.leadtime])

# sea_ice_area_binned.compute()

In [187]:
sea_ice_area_binned.shape

(90, 36)

In [172]:
longitude_bin = xr.DataArray(np.linspace(0, 360, 36+1))
sea_ice_area_binned = xr.apply_ufunc(sic_compute_np,
                                     xarr[f"sic_{0}"].groupby_bins("lon", longitude_bin),
                                     input_core_dims=[["leadtime", "time"]],
                                    #  output_core_dims=[[]],
                                    #  exclude_dims=set(["time", "stacked_yc_xc"]),
                                     dask="parallelized",
                                     output_dtypes=[float],
                                     dask_gufunc_kwargs={
                                         "output_sizes": {"sia": 36}
                                        }
                                     )
sea_ice_area_binned.compute()

[[[nan]
  [nan]
  [nan]
  ...
  [nan]
  [nan]
  [nan]]

 [[nan]
  [nan]
  [nan]
  ...
  [nan]
  [nan]
  [nan]]

 [[nan]
  [nan]
  [nan]
  ...
  [nan]
  [nan]
  [nan]]

 ...

 [[nan]
  [nan]
  [nan]
  ...
  [nan]
  [nan]
  [nan]]

 [[nan]
  [nan]
  [nan]
  ...
  [nan]
  [nan]
  [nan]]

 [[nan]
  [nan]
  [nan]
  ...
  [nan]
  [nan]
  [nan]]]


ValueError: applied function returned data with unexpected number of dimensions. Received 0 dimension(s) but expected 1 dimensions with names: ('stacked_yc_xc',)

In [134]:
import dask.array as da

def apply_bin_along_leadtime(func, arr):
    return da.apply_along_axis(func, arr=arr, axis=3)

# da.map_blocks(compute_sea_ice_area_binned, xarr_chunk[f"sic_{0}"], dtype=xarr_chunk[f"sic_{0}"].dtype).compute()

apply_bin_along_leadtime(compute_sea_ice_area_binned, xarr_chunk[f"sic_{0}"])

<class 'numpy.ndarray'>


AttributeError: 'numpy.ndarray' object has no attribute 'groupby_bins'